In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler

import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import json
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import random
import time
from collections import defaultdict
import xml.etree.ElementTree as ET
import yaml

print("=== Faster R-CNN Training Setup ===")
print(f'Python version: {sys.version.split()[0]}')
print(f'PyTorch version: {torch.__version__}')
print(f'Torchvision version: {torchvision.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')
    print(f'CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

=== Faster R-CNN Training Setup ===
Python version: 3.10.18
PyTorch version: 2.7.1+cu118
Torchvision version: 0.22.1+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU
CUDA memory: 4.3 GB


In [12]:
# Helper function to load data configuration from yaml_path
# Returns a dict with paths, number of classes, and class names
def load_data_config(yaml_path):
    yaml_path = Path(yaml_path)
    
    if not yaml_path.exists():
        raise FileNotFoundError(f"data.yaml not found at {yaml_path}")
    
    with open(yaml_path, 'r') as file:
        config = yaml.safe_load(file)
    
    print(f"Loaded data configuration from {yaml_path}")
    print(f"Number of classes: {config['nc']}")
    print(f"Class names: {config['names']}")
    
    return config

In [13]:
# Configuration class to store all hyperparameters and settings
class Config:
    DATA_ROOT = Path("../data")
    DATA_YAML = "../data/data.yaml"
    
    USE_SUBSET = None
    SUBSET_VALID = None
    
    # Load dataset configuration from YAML file
    try:
        data_config = load_data_config(DATA_YAML)
        NUM_CLASSES = data_config['nc'] + 1  # plus 1 for background class
        CLASS_NAMES = ['background'] + data_config['names']  # Add background as class 0
    except FileNotFoundError:
        print("Error: data.yaml not found")
    
    # Model configuration
    BACKBONE = 'resnet50'
    PRETRAINED = True
    
    # Training hyperparameters
    BATCH_SIZE = 6
    LEARNING_RATE = 0.002
    NUM_EPOCHS = 5
    WEIGHT_DECAY = 0.0005
    MOMENTUM = 0.9
    
    # Image processing
    IMG_SIZE = (416, 416)  # Input image size (height, width)
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]
    
    # Training settings
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    NUM_WORKERS = 0
    PRINT_FREQ = 20
    SAVE_FREQ = 1
    
    GRADIENT_ACCUMULATION_STEPS = 1
    MIXED_PRECISION = True
    
    # Learning rate scheduler settings
    LR_STEP_SIZE = 2
    LR_GAMMA = 0.3

# Create config instance
config = Config()
print(f"Training on device: {config.DEVICE}")
print(f"Number of classes: {config.NUM_CLASSES}")

Loaded data configuration from ..\data\data.yaml
Number of classes: 12
Class names: ['Ants', 'Bees', 'Beetles', 'Caterpillars', 'Earthworms', 'Earwigs', 'Grasshoppers', 'Moths', 'Slugs', 'Snails', 'Wasps', 'Weevils']
Training on device: cuda
Number of classes: 13


In [14]:

# DATASET CLASS FOR LOADING AGROPEST DATA
class AgroPestDataset(Dataset):
    """
    Custom Dataset class for loading AgroPest-12 data
    
    This class handles:
    - Loading images from the dataset
    - Converting YOLO format annotations to Faster R-CNN format
    - Applying data augmentations
    - Returning properly formatted tensors for training
    """
    
    def __init__(self, data_dir, split='train', transform=None):
        """
        Initialize the dataset
        
        Args:
            data_dir (str/Path): Root directory containing train/valid/test folders
            split (str): Which split to use ('train', 'valid', or 'test')
            transform (callable): Optional transform to apply to images
        """
        self.data_dir = Path(data_dir)
        self.split = split
        self.transform = transform
        
        # Set paths for images and labels
        self.image_dir = self.data_dir / split / 'images'
        self.label_dir = self.data_dir / split / 'labels'
        
        # Get list of all image files
        self.image_paths = list(self.image_dir.glob('*.jpg'))
        
        # Use subset for faster training/testing if specified
        if hasattr(config, 'USE_SUBSET') and config.USE_SUBSET is not None:
            if split == 'train' and config.USE_SUBSET:
                self.image_paths = self.image_paths[:config.USE_SUBSET]
                print(f"Using subset: {len(self.image_paths)} images (from {config.USE_SUBSET} requested)")
            elif split == 'valid' and hasattr(config, 'SUBSET_VALID') and config.SUBSET_VALID:
                self.image_paths = self.image_paths[:config.SUBSET_VALID]
                print(f"Using validation subset: {len(self.image_paths)} images")
                
        # Verify that image and label directories exist
        if not self.image_dir.exists():
            raise FileNotFoundError(f"Image directory not found: {self.image_dir}")
        if not self.label_dir.exists():
            raise FileNotFoundError(f"Label directory not found: {self.label_dir}")
            
        print(f"Loaded {len(self.image_paths)} images from {split} split")
    
    def __len__(self):
        """Return the total number of samples in the dataset"""
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """
        Get a single sample from the dataset
        
        Args:
            idx (int): Index of the sample to retrieve
            
        Returns:
            tuple: (image, target) where image is a tensor and target is a dict
        """
        
        img_path = self.image_paths[idx]
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Load annotations
        label_path = self.label_dir / (img_path.stem + '.txt')
        
        boxes, labels = self._load_yolo_annotations(label_path, image.size)
        
        # Prepare target dictionary (format required by Faster R-CNN)
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([idx]),
            'area': self._calculate_area(boxes),
            'iscrowd': torch.zeros((len(boxes),), dtype=torch.int64)
        }
        
        # Apply transforms if specified
        if self.transform:
            image = self.transform(image)
        
        return image, target
    
    def _load_yolo_annotations(self, label_path, img_size):
        """
        Load and convert YOLO format annotations to Faster R-CNN format
        
        YOLO format: class_id center_x center_y width height (normalized 0-1)
        Faster R-CNN format: [x_min, y_min, x_max, y_max] (absolute pixels)
        
        Args:
            label_path (Path): Path to the YOLO label file
            img_size (tuple): (width, height) of the image
            
        Returns:
            tuple: (boxes, labels) as lists
        """
        boxes = []
        labels = []
        
        img_width, img_height = img_size
        
        # Check if label file exists (some images might not have annotations)
        if not label_path.exists():
            # Return empty annotations for images without labels
            return [], []
        
        # Read YOLO format annotations
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            line = line.strip()
            if not line:  # Skip empty lines
                continue
                
            # Parse YOLO format: class_id center_x center_y width height
            parts = line.split()
            if len(parts) != 5:
                continue  # Skip malformed lines
                
            class_id = int(parts[0])
            center_x = float(parts[1])
            center_y = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Convert normalized coordinates to absolute coordinates
            center_x_abs = center_x * img_width
            center_y_abs = center_y * img_height
            width_abs = width * img_width
            height_abs = height * img_height
            
            # Convert center format to corner format
            x_min = center_x_abs - width_abs / 2
            y_min = center_y_abs - height_abs / 2
            x_max = center_x_abs + width_abs / 2
            y_max = center_y_abs + height_abs / 2
            
            # Ensure coordinates are within image bounds
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(img_width, x_max)
            y_max = min(img_height, y_max)
            
            # Only add valid boxes (with positive area)
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(class_id + 1)  # Add 1 because Faster R-CNN uses 1-indexed labels
        
        return boxes, labels
    
    def _calculate_area(self, boxes):
        """Calculate area of bounding boxes"""
        if len(boxes) == 0:
            return torch.zeros((0,), dtype=torch.float32)
        
        boxes_tensor = torch.as_tensor(boxes, dtype=torch.float32)
        return (boxes_tensor[:, 2] - boxes_tensor[:, 0]) * (boxes_tensor[:, 3] - boxes_tensor[:, 1])

In [15]:
# =============================================================================
# 5. DATA TRANSFORMS AND AUGMENTATION
# =============================================================================

def get_transforms(split='train'):
    """
    Get data transforms for training or validation
    
    Args:
        split (str): 'train' for training transforms, anything else for validation
        
    Returns:
        torchvision.transforms.Compose: Composed transforms
    """
    if split == 'train':
        # Training transforms with data augmentation
        return transforms.Compose([
            transforms.Resize(config.IMG_SIZE),  # Resize to fixed size
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color augmentation
            transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
            transforms.ToTensor(),  # Convert PIL image to tensor
            transforms.Normalize(mean=config.MEAN, std=config.STD)  # Normalize with ImageNet stats
        ])
    else:
        # Validation transforms without augmentation
        return transforms.Compose([
            transforms.Resize(config.IMG_SIZE),  # Resize to fixed size
            transforms.ToTensor(),  # Convert PIL image to tensor
            transforms.Normalize(mean=config.MEAN, std=config.STD)  # Normalize with ImageNet stats
        ])

In [16]:
# =============================================================================
# 6. MODEL CREATION FUNCTION
# =============================================================================

def create_faster_rcnn_model(num_classes):
    """
    Create a Faster R-CNN model with custom number of classes
    
    How Faster R-CNN works:
    1. Backbone (ResNet-50): Extracts features from input image
    2. FPN (Feature Pyramid Network): Combines features at different scales
    3. RPN (Region Proposal Network): Generates potential object locations
    4. ROI Head: Classifies proposals and refines bounding boxes
    
    Args:
        num_classes (int): Number of classes including background
        
    Returns:
        torch.nn.Module: Faster R-CNN model
    """
    # Load pre-trained Faster R-CNN model with ResNet-50 backbone and FPN
    # This model is pre-trained on COCO dataset (80 classes + background)
    from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

    if config.PRETRAINED:
        weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    else:
        weights = None

    model = fasterrcnn_resnet50_fpn(weights=weights)
    
    # Get the number of input features for the classifier
    # The classifier head takes features from the ROI pooling layer
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one for our number of classes
    # FastRCNNPredictor includes both classification and bounding box regression
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    print(f"Created Faster R-CNN model with {num_classes} classes")
    print(f"Backbone: ResNet-50 + FPN")
    print(f"Classifier input features: {in_features}")
    
    return model

In [17]:
# =============================================================================
# 7. COLLATE FUNCTION FOR DATALOADER
# =============================================================================

def collate_fn(batch):
    """
    Custom collate function to handle variable number of objects per image
    
    Faster R-CNN needs a special collate function because:
    - Different images have different numbers of objects
    - We can't stack tensors with different shapes
    - We need to return lists of images and targets
    
    Args:
        batch (list): List of (image, target) tuples
        
    Returns:
        tuple: (images, targets) as lists
    """
    images = []
    targets = []
    
    for image, target in batch:
        images.append(image)
        targets.append(target)
    
    return images, targets

In [18]:
# =============================================================================
# 8. TRAINING FUNCTIONS
# =============================================================================

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=5):
    """
    Train the model for one epoch
    """
    model.train()
    
    # Initialize mixed precision scaler
    if config.MIXED_PRECISION:
        try:
            scaler = GradScaler("cuda")
        except TypeError:
            scaler = GradScaler()
    else:
        scaler = None
    
    losses = []
    optimizer.zero_grad()
    
    for batch_idx, (images, targets) in enumerate(data_loader):
        # Move images and targets to device (GPU)
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Forward pass with mixed precision if enabled
        if config.MIXED_PRECISION and scaler is not None:
            try:
                with autocast("cuda"):
                    loss_dict = model(images, targets)
            except AssertionError as e:
                if "Expected target boxes to be a tensor" in str(e):
                    print(f"Skipping batch with empty annotations")
                    continue  # Skip this batch
                else:
                    raise e  # Re-raise if it's a different error
        else:
            loss_dict = model(images, targets)
        
        # Calculate total loss
        total_loss = sum(loss for loss in loss_dict.values())
        
        # Scale loss for gradient accumulation
        total_loss = total_loss / config.GRADIENT_ACCUMULATION_STEPS
        
        # Backward pass
        if config.MIXED_PRECISION and scaler is not None:
            scaler.scale(total_loss).backward()  # Scaled backward pass
        else:
            total_loss.backward()
        
        losses.append(total_loss.item() * config.GRADIENT_ACCUMULATION_STEPS)  # Store unscaled loss
        
        # Update weights every GRADIENT_ACCUMULATION_STEPS
        if (batch_idx + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
            if config.MIXED_PRECISION and scaler is not None:
                scaler.step(optimizer)  # Update weights with scaling
                scaler.update()  # Update scaler for next iteration
            else:
                optimizer.step()
            
            optimizer.zero_grad()  # Clear gradients
            
        if batch_idx % print_freq == 0 or batch_idx == 0:
            current_lr = optimizer.param_groups[0]['lr']
            memory_used = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
            print(f'Epoch [{epoch}], Batch [{batch_idx}/{len(data_loader)}], '
                  f'Loss: {total_loss.item() * config.GRADIENT_ACCUMULATION_STEPS:.4f}, '
                  f'LR: {current_lr:.6f}, GPU Memory: {memory_used:.2f}GB')
            
            # Print individual loss components for debugging (less frequently)
            if batch_idx % (print_freq * 2) == 0:
                for loss_name, loss_value in loss_dict.items():
                    print(f'  {loss_name}: {loss_value.item():.4f}')
        
        # Clear cache periodically to prevent memory fragmentation
        if batch_idx % 10 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    return np.mean(losses)

def validate_model(model, data_loader, device):
    """
    Validate the model (calculate validation loss)
    
    Args:
        model: Trained model
        data_loader: DataLoader with validation data
        device: Device to run validation on
        
    Returns:
        float: Average validation loss
    """
    model.train()  # Keep in training mode to get loss values
    
    losses = []
    
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for images, targets in data_loader:
            # Move to device
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Forward pass
            loss_dict = model(images, targets)
            total_loss = sum(loss for loss in loss_dict.values())
            
            losses.append(total_loss.item())
    
    return np.mean(losses)

In [19]:

# =============================================================================
# 9. MAIN TRAINING FUNCTION
# =============================================================================

def train_faster_rcnn():
    """
    Main function to train the Faster R-CNN model
    """
    # Create datasets
    print("Loading datasets...")
    train_dataset = AgroPestDataset(
        config.DATA_ROOT, 
        split='train',
        transform=get_transforms('train')
    )
    
    valid_dataset = AgroPestDataset(
        config.DATA_ROOT,
        split='valid', 
        transform=get_transforms('valid')
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,  # Shuffle training data
        num_workers=config.NUM_WORKERS,
        collate_fn=collate_fn  # Use custom collate function
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,  # Don't shuffle validation data
        num_workers=config.NUM_WORKERS,
        collate_fn=collate_fn
    )
    
    print(f"Train batches: {len(train_loader)}")
    print(f"Valid batches: {len(valid_loader)}")
    
    # Create model
    model = create_faster_rcnn_model(config.NUM_CLASSES)
    model.to(config.DEVICE)
    
    # Create optimizer
    # Using SGD with momentum (common choice for object detection)
    optimizer = optim.SGD(
        model.parameters(),
        lr=config.LEARNING_RATE,
        momentum=config.MOMENTUM,
        weight_decay=config.WEIGHT_DECAY
    )
    
    # Learning rate scheduler (reduce LR when training stagnates)
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config.LR_STEP_SIZE,  # Use config value
        gamma=config.LR_GAMMA          # Use config value
    )
    
    # Training loop
    best_loss = float('inf')
    train_losses = []
    valid_losses = []
    
    print(f"Starting training for {config.NUM_EPOCHS} epochs...")
    print(f"  - Batch size: {config.BATCH_SIZE} (with gradient accumulation: {config.GRADIENT_ACCUMULATION_STEPS})")
    print(f"  - Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - Image size: {config.IMG_SIZE}")
    print(f"  - Mixed precision: {config.MIXED_PRECISION}")
    print(f"  - Memory monitoring enabled")
    
    # Clear GPU cache before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        initial_memory = torch.cuda.memory_allocated() / 1e9
        print(f"  - Initial GPU memory: {initial_memory:.2f}GB / 4.3GB")
    
    for epoch in range(config.NUM_EPOCHS):
        print(f"\n=== Epoch {epoch + 1}/{config.NUM_EPOCHS} ===")
        epoch_start_time = time.time()

        # Train for one epoch
        train_loss = train_one_epoch(
            model, optimizer, train_loader, config.DEVICE, epoch + 1, config.PRINT_FREQ
        )
        
        # Validate
        valid_loss = validate_model(model, valid_loader, config.DEVICE)
        
        # Update learning rate
        scheduler.step()
        
        # Calculate epoch time
        epoch_time = time.time() - epoch_start_time
        
        # Record losses
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        # Memory monitoring for RTX 3050 Ti
        if torch.cuda.is_available():
            peak_memory = torch.cuda.max_memory_allocated() / 1e9
            current_memory = torch.cuda.memory_allocated() / 1e9
            torch.cuda.reset_peak_memory_stats()  # Reset for next epoch
        else:
            peak_memory = current_memory = 0
        
        print(f"Epoch {epoch + 1} Summary:")
        print(f"  Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
        print(f"  Time: {epoch_time:.1f}s, Peak GPU Memory: {peak_memory:.2f}GB")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Memory warning
        if peak_memory > 4.0:
            print(f"    WARNING: High memory usage ({peak_memory:.2f}GB). Consider reducing batch size or image size.")
        
        # Save best model
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'config': config.__dict__  # Save all config settings
            }, 'best_faster_rcnn_model.pth')
            print(f"  ✅ New best model saved! Validation loss: {best_loss:.4f}")
        
        # Save checkpoint every few epochs
        if (epoch + 1) % config.SAVE_FREQ == 0:
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'config': config.__dict__
            }, f'faster_rcnn_checkpoint_epoch_{epoch + 1}.pth')
    
    # Plot training curves    
    plt.close('all')
    
    # Create plot with error handling
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(train_losses, label='Training Loss', marker='o')
    ax.plot(valid_losses, label='Validation Loss', marker='s')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Faster R-CNN Training Progress')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Save plot
    plt.tight_layout()
    plt.savefig('../results/faster_rcnn_evaluation/faster_rcnn_training_curves.png', dpi=100, bbox_inches='tight')
    print(f"✅ Training curves saved in '../results/faster_rcnn_evaluation/faster_rcnn_training_curves.png'")
    
    plt.close()
    
    return model

In [20]:
# =============================================================================
# 11. MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    # Verify data directory and yaml file exist
    if not config.DATA_ROOT.exists():
        print(f"Error: Data directory not found at {config.DATA_ROOT}")
        sys.exit(1)
    
    yaml_path = Path(config.DATA_YAML)
    if not yaml_path.exists():
        print(f"Error: data.yaml not found at {yaml_path}")
        sys.exit(1)
    
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    
    # Start training
    print("Starting Faster R-CNN training for AgroPest-12 dataset")
    print(f"Configuration:")
    print(f"  Classes: {config.NUM_CLASSES}")
    print(f"  Batch size: {config.BATCH_SIZE}")
    print(f"  Learning rate: {config.LEARNING_RATE}")
    print(f"  Epochs: {config.NUM_EPOCHS}")
    print(f"  Device: {config.DEVICE}")
    
    # Train the model
    trained_model = train_faster_rcnn()
    
    print("\nTraining completed! You can now:")
    print("1. Check 'best_faster_rcnn_model.pth' for the best model")
    print("2. View 'faster_rcnn_training_curves.png' for training progress")
    print("3. Run inference using the trained model")

Starting Faster R-CNN training for AgroPest-12 dataset
Configuration:
  Classes: 13
  Batch size: 6
  Learning rate: 0.002
  Epochs: 5
  Device: cuda
Loading datasets...
Loaded 11502 images from train split
Loaded 1095 images from valid split
Train batches: 1917
Valid batches: 183
Created Faster R-CNN model with 13 classes
Backbone: ResNet-50 + FPN
Classifier input features: 1024
Starting training for 5 epochs...
  - Batch size: 6 (with gradient accumulation: 1)
  - Effective batch size: 6
  - Image size: (416, 416)
  - Mixed precision: True
  - Memory monitoring enabled
  - Initial GPU memory: 0.71GB / 4.3GB

=== Epoch 1/5 ===
Epoch [1], Batch [0/1917], Loss: 3.2734, LR: 0.002000, GPU Memory: 0.72GB
  loss_classifier: 2.8745
  loss_box_reg: 0.0445
  loss_objectness: 0.2587
  loss_rpn_box_reg: 0.0957
Epoch [1], Batch [20/1917], Loss: 0.1036, LR: 0.002000, GPU Memory: 0.89GB
Epoch [1], Batch [40/1917], Loss: 0.0910, LR: 0.002000, GPU Memory: 0.89GB
  loss_classifier: 0.0176
  loss_box_r